<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
import pandas as pd
url = 'https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv'
df = pd.read_csv(url)

## Let's first look at some basic info on the DF for cleaning

In [2]:
pd.set_option('display.max_columns', 25)

In [3]:
print(df.shape)
df.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [5]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

^ Nice

In [6]:
# Interesting that TotalCharges is an object...
df['TotalCharges'].value_counts(ascending=False)[:5]

         11
20.2     11
19.75     9
19.65     8
20.05     8
Name: TotalCharges, dtype: int64

In [7]:
# We have a blank value 11 times, my guess is that these are for people that have not
# Been charged yet, i.e. tenure=0, let's check
df[df['TotalCharges'] == ' ']

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,,No


In [8]:
new_totals = df[df['tenure'] == 0]['MonthlyCharges'].values
locs = df[df['tenure'] == 0].index.values
df['TotalCharges'].iloc[locs] = new_totals
df[df['tenure'] == 0]

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,52.55,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,20.25,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,80.85,No
1082,4367-NUYAO,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,25.75,No
1340,1371-DWPAZ,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,56.05,No
3331,7644-OMVMY,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,19.85,No
3826,3213-VVOLG,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,25.35,No
4380,2520-SGTTA,Female,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,20,No
5218,2923-ARZLG,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,19.7,No
6670,4075-WKNIU,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,73.35,No


In [9]:
df['TotalCharges'] = df['TotalCharges'].astype(float)
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [10]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692,2279.798992
std,0.368612,24.559481,30.090047,2266.730170
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,398.550000
50%,0.000000,29.000000,70.350000,1394.550000
75%,0.000000,55.000000,89.850000,3786.600000
max,1.000000,72.000000,118.750000,8684.800000


## Let's one hot encode the categorical variables and then scale everything

In [11]:
!pip install category_encoders

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [12]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import category_encoders as ce

## Don't want to encode the customer_id or the target!

In [13]:
drops = df.columns[[0, -2, -3]].tolist()
drops

['customerID', 'TotalCharges', 'MonthlyCharges']

In [14]:
x = df.drop(drops, axis=1)
x.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,Yes


In [15]:
enc_scale = make_pipeline(
    ce.OneHotEncoder(),
    StandardScaler()
)

X = enc_scale.fit_transform(x)

In [16]:
print(X.shape)
X

(7043, 45)


array([[ 1.00955867, -1.00955867, -0.43991649, ..., -0.52504733,
         0.60102348, -0.60102348],
       [-0.99053183,  0.99053183, -0.43991649, ..., -0.52504733,
         0.60102348, -0.60102348],
       [-0.99053183,  0.99053183, -0.43991649, ..., -0.52504733,
        -1.66382851,  1.66382851],
       ...,
       [ 1.00955867, -1.00955867, -0.43991649, ..., -0.52504733,
         0.60102348, -0.60102348],
       [-0.99053183,  0.99053183,  2.27315869, ..., -0.52504733,
        -1.66382851,  1.66382851],
       [-0.99053183,  0.99053183, -0.43991649, ..., -0.52504733,
         0.60102348, -0.60102348]])

In [17]:
y = df['TotalCharges'].values
y

array([  29.85, 1889.5 ,  108.15, ...,  346.45,  306.6 , 6844.5 ])

In [18]:
X.shape, y.shape

((7043, 45), (7043,))

^ Looks good! ^

## Let's fit a basic model before tinkering later

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [23]:
model = Sequential([
    Dense(20, input_dim = 45, activation='relu', name='In'),
    Dense(20, activation='relu', name='1'),
    Dense(1, name='Out')
])
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
In (Dense)                   (None, 20)                920       
_________________________________________________________________
1 (Dense)                    (None, 20)                420       
_________________________________________________________________
Out (Dense)                  (None, 1)                 21        
Total params: 1,361
Trainable params: 1,361
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Save to history for use later
history = model.fit(X,y, epochs=50, validation_split=.25)

Train on 5282 samples, validate on 1761 samples
Epoch 1/50
5282/5282 [==============================] - 1s 136us/sample - loss: 10271245.4211 - mean_squared_error: 10271247.0000 - val_loss: 10028365.0744 - val_mean_squared_error: 10028366.0000
Epoch 2/50
5282/5282 [==============================] - 0s 66us/sample - loss: 9183312.3708 - mean_squared_error: 9183313.0000 - val_loss: 7861124.3380 - val_mean_squared_error: 7861123.5000
Epoch 3/50
5282/5282 [==============================] - 0s 66us/sample - loss: 5787477.1600 - mean_squared_error: 5787479.0000 - val_loss: 3843552.0947 - val_mean_squared_error: 3843552.5000
Epoch 4/50
5282/5282 [==============================] - 0s 66us/sample - loss: 2347413.0849 - mean_squared_error: 2347413.0000 - val_loss: 1649889.5667 - val_mean_squared_error: 1649889.3750
Epoch 5/50
5282/5282 [==============================] - 0s 65us/sample - loss: 1252081.1991 - mean_squared_error: 1252081.1250 - val_loss: 1257522.1758 - val_mean_squared_error: 12575

In [25]:
first_df = pd.DataFrame(history.history).reset_index().rename({'index':'epoch'}, axis=1).set_index('epoch')
min_mse = first_df['val_mean_squared_error'].min()
print(f'Lowest Val MSE: \n${min_mse:,.02f}')

Lowest Val MSE: 
$18,122.82


## Let's do some gridsearching and connect to wandb

In [23]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [82]:
!pip install wandb

    100% |████████████████████████████████| 1.3MB 17.9MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/20/8c/4543981439d23c4ff65b2e62dddd767ebc84a8e664a9b67e840d1e2730d3/GitPython-3.0.5-py3-none-any.whl
    100% |████████████████████████████████| 92kB 27.6MB/s ta 0:00:01
    100% |████████████████████████████████| 256kB 39.7MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 33.9MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 37.0MB/s ta 0:00:01
    100% |████████████████████████████████| 102kB 38.2MB/s a 0:00:01
  Using cached https://files.pythonhosted.org/packages/03/6c/99296f89bad2ef85626e1df9f677acbee8885bb043ad82ad3ed4746d2325/gitdb2-2.0.6-py2.py3-none-any.whl
    100% |████████████████████████████████| 184kB 29.6MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/55/d2/866d45e3a121ee15a1dc013824d58072fd5c7799c9c34d01378eb262ca8f/smmap2-2.0.5-py2.py3-none-any.whl
  Running setup.py bdist_wheel for watchdog ...

In [36]:
import wandb
from wandb.keras import WandbCallback

In [34]:
!wandb login 46c02f2598d6b20efbf9a0ea2d6c3541debfbfe6

wandb: Appending key for api.wandb.ai to your netrc file: /home/ec2-user/.netrc
Successfully logged in to Weights & Biases!


In [37]:
wandb.init(project="assignment", entity="ds8")

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable


W&B Run: https://app.wandb.ai/ds8/assignment/runs/ew9qdyn7

In [38]:
seed = 42
np.random.seed(seed)

In [41]:
input_dim = X.shape[1]

# Let's use the same model as above and gridsearch epochs/batch size
def create_model():
    model = Sequential([
        Dense(50, input_dim=input_dim, activation='relu', name='In'),
        Dense(50, activation='relu', name='1'),
        Dense(1, name='Out')
    ])
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    return model

model = KerasRegressor(build_fn=create_model, verbose=True)

param_grid = {
    'batch_size': (10, 100),
    'epochs': (50, 100)
}

search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
search_result = search.fit(X, y)

Epoch 1/50
4695/4695 [==============================] - 2s 340us/sample - loss: 6400373.8032 - mean_squared_error: 6400374.5000
Epoch 2/50
4695/4695 [==============================] - 1s 233us/sample - loss: 865288.0843 - mean_squared_error: 865288.3125
Epoch 3/50
4695/4695 [==============================] - 1s 238us/sample - loss: 553593.3053 - mean_squared_error: 553593.3125
Epoch 4/50
4695/4695 [==============================] - 1s 234us/sample - loss: 360955.7425 - mean_squared_error: 360955.6875
Epoch 5/50
4695/4695 [==============================] - 1s 235us/sample - loss: 219840.8257 - mean_squared_error: 219840.7031
Epoch 6/50
4695/4695 [==============================] - 1s 235us/sample - loss: 122272.3060 - mean_squared_error: 122272.3359
Epoch 7/50
4695/4695 [==============================] - 1s 236us/sample - loss: 67302.2687 - mean_squared_error: 67302.2500
Epoch 8/50
4695/4695 [==============================] - 1s 236us/sample - loss: 42455.2843 - mean_squared_error: 42455

In [43]:
search_result.cv_results_

{'mean_fit_time': array([ 58.59903709, 174.03383048,  50.87354112,  43.51453106]),
 'std_fit_time': array([ 0.28854528, 81.59802384,  6.82596435, 38.30680456]),
 'mean_score_time': array([1.15443707, 1.19968526, 3.86835742, 1.09834472]),
 'std_score_time': array([0.01915375, 0.07049878, 1.18692615, 0.00522086]),
 'param_batch_size': masked_array(data=[10, 10, 100, 100],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_epochs': masked_array(data=[50, 100, 50, 100],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'batch_size': 10, 'epochs': 50},
  {'batch_size': 10, 'epochs': 100},
  {'batch_size': 100, 'epochs': 50},
  {'batch_size': 100, 'epochs': 100}],
 'split0_test_score': array([ -8546.54412098,  -8644.01404994, -47911.94999734, -12969.59339232]),
 'split1_test_score': array([ -8958.21000461,  -8562.66397349, -61944.46415833, -10957.08052232]),
 'split2_test

In [46]:
search_result.best_params_

{'batch_size': 10, 'epochs': 50}

^ All that for the same batch_size and epochs as we had originally...

# I think I was supposed to use churn as the target...

In [26]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [38]:
df['Churn'] = df['Churn'].replace({'Yes':1, 'No':0})
df['Churn'].value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [39]:
new_drops = df.columns[[0, -1]]
x = df.drop(new_drops, axis=1)
x.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65


In [41]:
pipe = make_pipeline(
    ce.OneHotEncoder(),
    StandardScaler()
)
X = pipe.fit_transform(x)
y = df['Churn']

In [43]:
X.shape, y.shape

((7043, 45), (7043,))

In [86]:
model = Sequential([
    Dense(20, input_dim = 45, activation='relu', name='In'),
    Dense(20, activation='sigmoid', name='1'),
    Dense(1, activation='sigmoid', name='Out')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
In (Dense)                   (None, 20)                920       
_________________________________________________________________
1 (Dense)                    (None, 20)                420       
_________________________________________________________________
Out (Dense)                  (None, 1)                 21        
Total params: 1,361
Trainable params: 1,361
Non-trainable params: 0
_________________________________________________________________


In [87]:
model.fit(X,y,verbose=True, epochs=20, validation_split=.25)

Train on 5282 samples, validate on 1761 samples
Epoch 1/20
5282/5282 [==============================] - 5s 1ms/sample - loss: 0.4757 - acc: 0.7702 - val_loss: 0.4558 - val_acc: 0.7893
Epoch 2/20
5282/5282 [==============================] - 3s 505us/sample - loss: 0.4273 - acc: 0.7927 - val_loss: 0.4441 - val_acc: 0.7978
Epoch 3/20
5282/5282 [==============================] - 3s 496us/sample - loss: 0.4172 - acc: 0.7984 - val_loss: 0.4419 - val_acc: 0.7990
Epoch 4/20
5282/5282 [==============================] - 3s 563us/sample - loss: 0.4126 - acc: 0.8025 - val_loss: 0.4376 - val_acc: 0.7961
Epoch 5/20
5282/5282 [==============================] - 3s 489us/sample - loss: 0.4097 - acc: 0.8069 - val_loss: 0.4356 - val_acc: 0.7933
Epoch 6/20
5282/5282 [==============================] - 3s 566us/sample - loss: 0.4082 - acc: 0.8099 - val_loss: 0.4362 - val_acc: 0.8024
Epoch 7/20
5282/5282 [==============================] - 3s 519us/sample - loss: 0.4066 - acc: 0.8065 - val_loss: 0.4343 - val_

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?